In [1]:
#%matplotlib inline
from collections import defaultdict, OrderedDict
from gensim.models import Word2Vec
from gensim.models.phrases import Phraser
import matplotlib.pyplot as plt
import pandas as pd
import os
import re

In [2]:
from util_kw import *

In [3]:
path_root_processed='./processedText'
path_models = './word2vec_models'
path_phraser_models='./phraser_models'

journal = 'IN-indianexpress-editorials' 
mypath_processed = path_root_processed +'/'+journal 

wvm = Word2Vec.load(path_models+'/'+journal+'_with_phraser').wv
vocab = wvm.vocab
all_words = [x for x in vocab.keys()]

In [28]:
def count_words(myp_p, journ, lista):
    fpath = path_phraser_models + '/' + journ + '_00_bigramphraser'
    print (fpath)
    
    bigram = Phraser.load(fpath)
    texts = {}
    
    for fname in os.listdir(myp_p)[:18]:
        print ("Doing "+fname)
        text = []
        for line in open(myp_p + '/' + fname, 'r'):
            text = text + bigram[line.split()]
        
        print (len(text))
        
        frequency = defaultdict(float)
        for word in text:
            frequency[word] += 1
        
        texts[fname] = frequency
    
    word_count = {}   
    
    for fname in texts.keys():
        count = 0.0
        for word in lista.keys() :
            if word in texts[fname].keys() :
                count = count + texts[fname][word] * lista[word]
        word_count[fname] = count
        
    return word_count

In [29]:
def generate_time_series(l_words):
    wc = count_words(mypath_processed, journal, l_words)
    
    wc_summary = defaultdict(float)
    for d in wc.keys() :
        m = re.search('([0-9\-]+)\_', d)
        wc_summary[pd.to_datetime(m.group(1))] += wc[d]

    wc_ordered = OrderedDict(sorted(wc_summary.items()))
    ts_ret = pd.Series(wc_ordered.values(), index=wc_ordered.keys())
    return ts_ret

In [39]:
ts1 = generate_time_series(get_words(wvm, 'violence'))
ts2 = generate_time_series(get_words(wvm, 'muslim'))

./phraser_models/IN-indianexpress-editorials_00_bigramphraser
Doing 2017-8-2_18h30m2s__-1427363902.html.txt
467
Doing 2016-7-4_0h39m0s__1639791264.html.txt
264
Doing 2016-8-8_18h42m0s__-444715807.html.txt
226
Doing 2016-11-23_18h48m2s__542060785.html.txt
354
Doing 2016-5-31_19h0m1s__-1760088565.html.txt
316
Doing 2016-4-29_0h41m4s__1281894667.html.txt
356
Doing 2017-4-14_18h40m4s__535637875.html.txt
372
Doing 2017-9-1_18h59m4s__-2069223554.html.txt
451
Doing 2017-7-14_19h8m4s__-1278922685.html.txt
423
Doing 2017-11-24_19h4m4s__-757061614.html.txt
391
Doing 2016-7-14_18h55m3s__1576677172.html.txt
256
Doing 2017-6-1_20h23m3s__237053542.html.txt
423
Doing 2016-5-27_0h1m4s__-1969476674.html.txt
365
Doing 2016-4-6_0h5m2s__-1267849646.html.txt
366
Doing 2017-11-10_18h45m4s__813377562.html.txt
280
Doing 2016-4-6_0h5m2s__1502093212.html.txt
384
Doing 2016-9-9_19h0m4s__-1642910183.html.txt
300
Doing 2016-7-19_18h45m1s__906045086.html.txt
333
./phraser_models/IN-indianexpress-editorials_00_bigra

In [42]:
ts1.corr(ts2)

TypeError: unsupported operand type(s) for +: 'odict_values' and 'odict_values'

In [13]:
txt = '(correlation: '+ str(ts1.corr(ts2,method='pearson')) +') journal: '+ journal
plt.figure(figsize=(20,4))
ts1.plot(label='dalits')
ts2.plot(label='violence')
#plt.plot_date(x=wc_ordered.keys(), y=wc_ordered.values(), fmt="r-")
plt.title("Timeline plot "+ txt)
plt.legend(loc='upper left')
plt.ylabel("Word count")
plt.grid(True)
plt.show()

TypeError: unsupported operand type(s) for +: 'odict_values' and 'odict_values'